In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('../input/sf-crime/train.csv.zip')
train_data.head()

In [ ]:
sample_submission = pd.read_csv('../input/sf-crime/sampleSubmission.csv.zip')

In [ ]:
test_data = pd.read_csv('../input/sf-crime/test.csv.zip')
test_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data['year'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[0])
train_data['month'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[1])
train_data['day'] = train_data['Dates'].apply(lambda x : x.split()[0].split('-')[2])
time = train_data['Dates'].apply(lambda x : x.split()[1].split(':')[:2])
train_data.head()

In [ ]:
train_data['time'] = [int(x[0])*60 + int(x[1]) for x in time]
train_data.drop('Dates', inplace=True, axis=1)

In [ ]:
train_data.head()

In [ ]:
test_data['year'] =test_data['Dates'].apply(lambda x : x.split()[0].split('-')[0])
test_data['month'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[1])
test_data['day'] = test_data['Dates'].apply(lambda x : x.split()[0].split('-')[2])
time = test_data['Dates'].apply(lambda x : x.split()[1].split(':')[:2])
test_data['time'] = [int(x[0])*60 + int(x[1]) for x in time]
test_data.drop('Dates', inplace=True, axis=1)
test_data.head()

In [ ]:
train_data['DayOfWeek'].replace(to_replace=['Monday', "Tuesday", 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                               value=[i for i in range(0,7)], inplace=True)
test_data['DayOfWeek'].replace(to_replace=['Monday', "Tuesday", 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                               value=[i for i in range(0,7)], inplace=True)

In [ ]:
test_data.head()

In [ ]:
train_data = pd.concat([train_data,pd.get_dummies(train_data['PdDistrict'])],axis = 1)
train_data.drop('PdDistrict',axis = 1, inplace = True)
train_data.head()

In [ ]:
test_data = pd.concat([test_data,pd.get_dummies(test_data['PdDistrict'])],axis = 1)
test_data.drop('PdDistrict',axis = 1, inplace = True)
test_data.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
std_sc = StandardScaler()
std_sc.fit(train_data[["X","Y","time"]]) 
train_data[["X","Y","time"]]  = std_sc.transform(train_data[["X","Y","time"]])
train_data.head()

In [ ]:
std_sc.fit(test_data[["X","Y","time"]]) 
test_data[["X","Y","time"]]  = std_sc.transform(test_data[["X","Y","time"]])
test_data.head()

In [ ]:
train_data.drop(['Descript','Resolution','Address'],axis = 1, inplace = True)
test_data.drop('Address',axis = 1, inplace = True)
print(train_data.shape)
print(test_data.shape)

In [ ]:
X = train_data.drop('Category',axis = 1)
y = pd.get_dummies(train_data['Category'])
y.head()

In [ ]:
X = X.astype(float)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1], )))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(39, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
         verbose=2,
         batch_size=128,
         epochs=32,
         validation_data=(X_test, y_test))

In [ ]:
test=test_data.drop(['Id'],axis=1)
test=test.astype(float)
test.dtypes

In [ ]:
pred=model.predict(test)

In [ ]:
m = np.max(pred, axis=1).reshape(-1, 1)
predicted = np.array((pred == m), dtype='int32')
predicted

In [ ]:
col_names=list(sample_submission.columns)
col_names.remove('Id')
print(col_names)

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test_data['Id']
for i , entry in enumerate(col_names):
    submission[entry] = predicted[:,i]

In [ ]:
submission.to_csv('../working/submission.csv', index=False)